<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/restructure_code/ExploringGamePars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [ ]:
!pip install procgen

In [ ]:
#Clone git
!git clone -b restructure_code https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

In [ ]:
#update git
%cd /content/my_project
! git pull

In [ ]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

# TESTING

In [ ]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO


## Create model

In [ ]:
#The hyperparameters for the model is set in the hyperparameter module.

#choose a baseline
set_hyperparameters(baseline='Procgen')

#Or set manually
h.version="Test"
h.time_limit_hours=0
h.time_limit_minutes=10

#create model using current configuration in h (creating a model will also create log files on drive)
# model = PPO(print_output=True)


## Train model

In [ ]:
# policy = model.train()

In [ ]:
tools.create_index_table_from_txt_files()

In [ ]:
from utils import make_env
env = make_env(32, num_levels=10)
obs = env.reset()

model = PPO(print_output=True)
policy = model.policy
action, log_prob, value = policy.act(obs)
next_obs, reward, done, info = env.step(action)

In [39]:
obs = env.reset()


for step in range(128):
    # Use policy
    action, log_prob, value = policy.act(obs)
    
    # Take step in environment
    next_obs, reward, done, info = env.step(action)

    #print
    # print("Step {}".format(step))
    if True in done:
      print("[{}] Environments {} are done".format(step, [index for index in range(len(done)) if done[index] == True]))
    for i in range(len(reward)):
      if reward[i] != 0:
        print("[{}] Environment {} got reward of {}".format(step, i, reward[i]))
    

    # Update current observation
    obs = next_obs


[0] Environments [31] are done
[0] Environment 3 got reward of 1.4214231967926025
[0] Environment 9 got reward of 1.4214231967926025
[0] Environment 17 got reward of 1.4214231967926025
[1] Environment 28 got reward of 1.4234546422958374
[2] Environment 19 got reward of 1.424985408782959
[2] Environment 28 got reward of 1.424985408782959
[5] Environment 8 got reward of 1.4291764497756958
[5] Environment 17 got reward of 1.4291764497756958
[6] Environments [24] are done
[6] Environment 24 got reward of 1.4302071332931519
[7] Environment 3 got reward of 1.430940866470337
[8] Environment 29 got reward of 1.4315505027770996
[8] Environment 30 got reward of 1.4315505027770996
[9] Environments [16] are done
[10] Environment 9 got reward of 1.4324977397918701
[11] Environment 19 got reward of 1.4324533939361572
[12] Environments [29] are done
[13] Environments [1] are done
[14] Environment 28 got reward of 1.4321144819259644
[16] Environment 28 got reward of 1.4303642511367798
[17] Environment

In [86]:
h.num_envs=2
model = PPO(print_output=True)

Observation space: Box(0.0, 1.0, (3, 64, 64), float32)
Action space: 15


In [112]:
import numpy as np

def evaluate_policy(model, 
                    nr_of_levels,
                    print_output=True):
  """
  TODO: Add Video generation
  """
  policy = model.policy

  #pick levels we did not train on. 
  eval_env = utils.make_env(model.num_envs, start_level=model.num_levels, num_levels=nr_of_levels)
  obs = eval_env.reset()

  #book-keeping
  completed_envs= []
  counter_compl_envs = np.zeros(model.num_envs)
  episode_rewards = np.zeros(model.num_envs)  #current episode rewards
  rewards = {}
  for i in range(model.num_envs):
    rewards[i] = []
  step_counter = 0

  policy.eval()
  while True:

    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    
    #if any reward, update envs still not done
    for i in range(len(reward)):
      if reward[i] != 0 and i not in completed_envs:
        episode_rewards[i] += reward[i]
    
    # If new environment done, complete it
    for i in [index for index in range(len(done)) if done[index] == True]:
      if i not in completed_envs:
        counter_compl_envs[i] += 1
        if print_output:
          print("Environment {:2d} completed its {:4d}th level at timestep {:6d} with a reward of {:10f}".format(i, int(counter_compl_envs[i]), step_counter, episode_rewards[i]))
        rewards[i].append(episode_rewards[i])
        episode_rewards[i] = 0
        if counter_compl_envs[i] == nr_of_levels:
          completed_envs.append(i)  
        
 

    # If all environments are done, break
    if len(completed_envs) == model.num_envs:
      break
    step_counter +=1
  # end while
  
  # Calculate average return
  total_reward = []
  for key, value in rewards.items():
    total_reward.append(sum(value))
  total_reward = np.mean(total_reward)/nr_of_levels

  if print_output:
    print('Average return:', total_reward)

  return total_reward, episode_rewards



In [113]:
import time
start = time.time()
total_reward, episode_rewards = evaluate_policy(model, 1)
print(time.time()-start)

Environment  1 completed its    1th level at timestep    135 with a reward of  14.658671
Environment  0 completed its    1th level at timestep    136 with a reward of  11.875379
Average return: 13.267025083303452
0.6281616687774658


In [115]:
start = time.time()
total_reward, episode_rewards = evaluate_policy(model, 2)
print(time.time()-start)

Environment  0 completed its    1th level at timestep     91 with a reward of   1.989494
Environment  1 completed its    1th level at timestep    135 with a reward of  17.390955
Environment  0 completed its    2th level at timestep    179 with a reward of   1.148283
Environment  1 completed its    2th level at timestep    254 with a reward of   2.908285
Average return: 5.859254255890846
1.1824548244476318


In [116]:
start = time.time()
total_reward, episode_rewards = evaluate_policy(model, 4)
print(time.time()-start)

Environment  1 completed its    1th level at timestep     80 with a reward of   0.000000
Environment  0 completed its    1th level at timestep    113 with a reward of  18.180873
Environment  0 completed its    2th level at timestep    186 with a reward of   1.596141
Environment  1 completed its    2th level at timestep    248 with a reward of   3.190590
Environment  0 completed its    3th level at timestep    281 with a reward of   1.610836
Environment  1 completed its    3th level at timestep    385 with a reward of   3.380248
Environment  0 completed its    4th level at timestep    433 with a reward of   8.308644
Environment  1 completed its    4th level at timestep    459 with a reward of   1.328211
Average return: 4.699442803859711
2.083005428314209
